### importing libraries

In [1]:
import numpy as np
import pandas as pd
import csv

### Reading Ball by Ball

In [2]:
df_ball=pd.read_csv('Ball_by_Ball.csv')
df_ball.head(5)

,Match_Id,Innings_Id,Over_Id,Ball_Id,Team_Batting_Id,Team_Bowling_Id,Striker_Id,Striker_Batting_Position,Non_Striker_Id,Bowler_Id,Batsman_Scored,Extra_Type,Extra_Runs,Player_dissimal_Id,Dissimal_Type,Fielder_Id
0,335987,1,1,1,1,2,1,1,2,14,0,legbyes,1,,,
1,335987,1,1,2,1,2,2,2,1,14,0,,,,,
2,335987,1,1,3,1,2,2,2,1,14,0,wides,1,,,
3,335987,1,1,4,1,2,2,2,1,14,0,,,,,
4,335987,1,1,5,1,2,2,2,1,14,0,,,,,


### Reading players

In [3]:
df_player=pd.read_csv('Player.csv')
df_player.head(10)

,Player_Id,Player_Name,DOB,Batting_Hand,Bowling_Skill,Country,Is_Umpire,Unnamed: 7
0,1,SC Ganguly,8-Jul-72,Left_Hand,Right-arm medium,India,0,NaN
1,2,BB McCullum,27-Sep-81,Right_Hand,Right-arm medium,New Zealand,0,NaN
2,3,RT Ponting,19-Dec-74,Right_Hand,Right-arm medium,Australia,0,NaN
3,4,DJ Hussey,15-Jul-77,Right_Hand,Right-arm offbreak,Australia,0,NaN
4,5,Mohammad Hafeez,17-Oct-80,Right_Hand,Right-arm offbreak,Pakistan,0,NaN
5,6,R Dravid,11-Jan-73,Right_Hand,Right-arm offbreak,India,0,NaN
6,7,W Jaffer,16-Feb-78,Right_Hand,Right-arm offbreak,India,0,NaN
7,8,V Kohli,5-Nov-88,Right_Hand,Right-arm medium,India,0,NaN
8,9,JH Kallis,16-Oct-75,Right_Hand,Right-arm fast-medium,South Africa,0,NaN
9,10,CL White,18-Aug-83,Right_Hand,Legbreak googly,Australia,0,NaN


### Creating dictionary conatining players and corresponding balls played

In [4]:
player_balls={}
for ball in range(0,len(df_ball)):
    if df_ball.iloc[ball]['Striker_Id'] in player_balls:
        player_balls[df_ball.iloc[ball]['Striker_Id']].append(ball)
    else:
        player_balls[df_ball.iloc[ball]['Striker_Id']]=[ball]

### Creating dictionary containing players and the runs they made

In [5]:
player_runs={}
player_fours={}
player_sixes={}
runs=0
for player in player_balls.keys():
    runs=0
    fours=0
    sixes=0
    for balls in player_balls[player]:
        if df_ball.iloc[balls]['Batsman_Scored'].isdigit():
            runs=runs+int(df_ball.iloc[balls]['Batsman_Scored'])
            if int(df_ball.iloc[balls]['Batsman_Scored'])==4:
                fours=fours+1
            if int(df_ball.iloc[balls]['Batsman_Scored'])==6:
                sixes=sixes+1
    player_runs[player]=runs
    player_fours[player]=fours
    player_sixes[player]=sixes

### Calculating fifties and hundreds and highest score

In [6]:
player_fifties={}
player_hundreds={}
player_highest={}
for player in player_balls.keys():
    mid1=-1
    runs=0
    player_fifties[player]=0
    player_hundreds[player]=0
    highest=0
    for balls in player_balls[player]:
        mid2=int(df_ball.iloc[balls]['Match_Id'])
        if mid1!=mid2:
            if runs>=50 and runs<100:
                player_fifties[player]+=1
            elif runs>=100:
                player_hundreds[player]+=1
            if runs>highest:
                highest=runs
            mid1=mid2
            if df_ball.iloc[balls]['Batsman_Scored'].isdigit():
                runs=int(df_ball.iloc[balls]['Batsman_Scored'])
            else:
                runs=0
        else:
            if df_ball.iloc[balls]['Batsman_Scored'].isdigit():
                runs+=int(df_ball.iloc[balls]['Batsman_Scored'])
    if runs>=50 and runs<100:
        player_fifties[player]+=1
    elif runs>=100:
        player_hundreds[player]+=1
    if runs>highest:
        highest=runs
    player_highest[player]=highest

### Calculating strike rate and average 

In [7]:
player_strikerate={}
player_average={}
player_dis={}
for balls in range(0,len(df_ball)):
    if df_ball.iloc[balls]['Player_dissimal_Id'].isdigit():
        if int(df_ball.iloc[balls]['Player_dissimal_Id']) in player_dis.keys():
            player_dis[int(df_ball.iloc[balls]['Player_dissimal_Id'])]+=1
        else:
            player_dis[int(df_ball.iloc[balls]['Player_dissimal_Id'])]=1 

In [8]:
for player in player_runs.keys():
    if len(player_balls[player])==0:
        player_strikerate[player]=0
        player_average[players]=0
    elif player in player_dis.keys():
        player_strikerate[player]=player_runs[player]/len(player_balls[player])
        player_average[player]=player_runs[player]/player_dis[player]
    else:
        player_strikerate[player]=player_runs[player]/len(player_balls[player]) 
        player_average[player]=player_runs[player]

### Calculating number of matches

In [9]:
player_nmatches={}
for player in player_balls.keys():
    matches=[]
    for ball in player_balls[player]:
        matches.append(int(df_ball.iloc[ball]['Match_Id']))
    player_nmatches[player]=len(set(matches))
        

### Creating table for batsman

In [10]:
df_batsman=pd.DataFrame()
df_batsman['Runs']=[player_runs[i] for i in player_runs.keys()]
df_batsman['Innings']=[player_nmatches[i] for i in player_nmatches.keys()]
df_batsman['Highest']=[player_highest[i] for i in player_highest.keys()]
df_batsman['Sixes']=[player_sixes[i] for i in player_sixes.keys()]
df_batsman['Fours']=[player_fours[i] for i in player_fours.keys()]
df_batsman['Average']=[round(player_average[i],2) for i in player_average.keys()]
df_batsman['Strike_Rate']=[round(player_strikerate[i]*100,2) for i in player_strikerate.keys()]
df_batsman['Fifties']=[player_fifties[i] for i in player_fifties.keys()]
df_batsman['Hundreds']=[player_hundreds[i] for i in player_hundreds.keys()]

In [11]:
df_batsman.head(5)

,Runs,Innings,Highest,Sixes,Fours,Average,Strike_Rate,Fifties,Hundreds
0,1349,56,91,42,137,35.50,101.73,7,0
1,2435,92,158,106,248,32.04,124.62,11,2
2,91,9,28,2,5,22.75,70.54,0,0
3,1322,61,71,60,90,41.31,120.07,5,0
4,64,8,16,2,7,9.14,76.19,0,0


In [12]:
merged=df_player.combine_first(df_batsman)
merged.to_csv('Batsman.csv')

### Converting to SQL Tables

In [31]:
import MySQLdb
df2 = pd.read_csv('Batsman.csv')
df2.head()
df2 = df2.drop(columns = ['Unnamed: 0', 'Bowling_Skill', 'Is_Umpire', 'Unnamed: 7'])
df2.head()

,Average,Batting_Hand,Country,DOB,Fifties,Fours,Highest,Hundreds,Innings,Player_Id,Player_Name,Runs,Sixes,Strike_Rate
0,35.50,Left_Hand,India,8-Jul-72,7.0,137.0,91.0,0.0,56.0,1,SC Ganguly,1349.0,42.0,101.73
1,32.04,Right_Hand,New Zealand,27-Sep-81,11.0,248.0,158.0,2.0,92.0,2,BB McCullum,2435.0,106.0,124.62
2,22.75,Right_Hand,Australia,19-Dec-74,0.0,5.0,28.0,0.0,9.0,3,RT Ponting,91.0,2.0,70.54
3,41.31,Right_Hand,Australia,15-Jul-77,5.0,90.0,71.0,0.0,61.0,4,DJ Hussey,1322.0,60.0,120.07
4,9.14,Right_Hand,Pakistan,17-Oct-80,0.0,7.0,16.0,0.0,8.0,5,Mohammad Hafeez,64.0,2.0,76.19


In [38]:
mydb = MySQLdb.connect(host = 'localhost', user = 'root', passwd = '', db = 'IPL', autocommit = 'true')
cursor = mydb.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS Batsman(Average int, Batting_Hand varchar(20), Country varchar(20), DOB varchar(20),Fifties int, Fours int, Highest int, Hundreds int, Innings int, Player_Id int primary key, Name varchar(20), Runs int,Sixes int, Strike_Rate double")

In [40]:
for i in range(0,len(df2)):
    row = list(df2.iloc[i])
#     print(row)
    sql = "INSERT INTO Batsman VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    cursor.execute(sql, row)

<class 'numpy.float64'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.int64'>
<class 'str'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>


IntegrityError: (1062, "Duplicate entry '1' for key 'PRIMARY'")